## LDA-TM : 
### Extract Topic Model with LdaModel from KCC News and CIS test data

In [1]:
import tensorflow as tf
import keras
#from tensorflow import keras
import pandas as pd
import numpy as np
import gc
import math
from playsound import playsound
#playsound("sound.mp3")
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
# 載入 ckiptagger 自訂詞典
User_Dict = {}
with open("KccLy2Dict2021.txt","r", encoding='utf-8-sig') as UDicts: 
    for udic in UDicts:
        udWord = udic.strip().split(" ")
        if len(udWord) > 1:
            User_Dict[udWord[0]] = udWord[1]
        else:
            User_Dict[udWord[0]] = 150    # 未給定權重者一律賦予預設值 150                
dictionary = construct_dictionary(User_Dict)

# 載入模型
ws = WS("./data")
#pos = POS("./data")
#ner = NER("./data")

# 載入 StopWord
stopword_list = []
with open("KccStopWord2020.txt","r", encoding='utf-8-sig') as stopwords:
    for stopword in stopwords:
        stopword_list.append(stopword.strip())

/home/abner0908/anaconda2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/abner0908/anaconda2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/abner0908/anaconda2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/abner0908/anaconda2/lib/python3.7/site-packages/tensorflow/python/f

In [2]:
#h
zhao = pd.read_csv('林岱樺_leg.csv',encoding="utf-8-sig") #匯該立委立院發言資料
zhao 

,屆,會期,日期,委員,逐字稿
0,9,1,3/7/2016,林岱樺,<U+FEFF>林委員岱樺：主席、各位列席官員、各位同仁。對於目前的經濟狀況，相信部長應該是...
1,9,1,3/9/2016,林岱樺,<U+FEFF>林委員岱樺：主席、各位列席官員、各位同仁。在進入今天的主題之前，我先請教你一...
2,9,1,3/14/2016,林岱樺,<U+FEFF>林委員岱樺：主席、各位列席官員、各位同仁。本席再延續針對國發基金的運作狀況，...
3,9,1,3/16/2016,林岱樺,<U+FEFF>林委員岱樺：主席、各位列席官員、各位同仁。現在農委會的政策有沒有要開放酪農進...
4,9,1,3/21/2016,林岱樺,<U+FEFF>林委員岱樺：主席、各位列席官員、各位同仁。在進入主題前，本席先針對電價問題來...
...,...,...,...,...,...
123,9,8,9/23/2019,林岱樺,<U+FEFF>林委員岱樺：（9時41分）主席、各位列席官員、各位同仁。本席針對三個議題、兩...
124,9,8,10/2/2019,林岱樺,<U+FEFF>林委員岱樺：（9 時 22 分）主席、各位列席官員、各位同仁。本席肯定部長在...
125,9,8,10/2/2019,林岱樺,<U+FEFF>林委員岱樺：（12 時 52 分）主席、各位列席官員、各位同仁。我現在要讓部...
126,9,8,10/14/2019,林岱樺,<U+FEFF>林委員岱樺：（9時9分）主席、各位列席官員、各位同仁。部長，本席今天要替業者...


In [3]:
str_str= list(zhao['逐字稿'])
str_str =(','.join('%s' %id for id in str_str)) 
# 開始斷詞
# sentence_segmentation = True,  => To consider delimiters
# segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
# recommend_dictionary = dictionary1, # words in this dictionary are encouraged
# coerce_dictionary = dictionary2, # words in this dictionary are forced
strCut = ws([str_str], sentence_segmentation=True,
                 segment_delimiter_set = {'"',"\r\n","，","「","」",",","？",'\n', "。", ":", "?", "!", ";", "、"},
                 coerce_dictionary = dictionary)

print("------------CKIPTagger Word Segment ws(str,....) + 刪除標點及停用字 ---------------")
stayed_lst_ly= []
for word in strCut[0]:
     # 刪除停用字
    if word not in stopword_list:
        stayed_lst_ly.append(word)
        
print(" / ".join(stayed_lst_ly))

------------CKIPTagger Word Segment ws(str,....) + 刪除標點及停用字 ---------------
林委員岱樺 / 列席 / 官員 / 位 / 同仁 / 目前 / 經濟 / 狀況 / 部長 / 
 / 席 / 簡報 / 第 1  / 頁 / 第 2  / 頁 / 現況 / 再度 / 提出來 / 馬 / 政府 / 主政 / 臺灣 / 經濟 / 
 / 狀況 / 張 / 圖表 / 經濟 / 平均 / 年 / 成長率 /  2.9% / 馬 / 總統 /  633 /   / 政見 / 
 / 經濟 / 成長率 /  6% / 跳票 / 回顧 / 民進黨 / 執政 / 經濟 / 成 / 
 / 長率 / 平均 /  4.4% /  2013 /  年 / 臺灣 / 經濟 / 成長率 / 數據 / 鄰近 / 國家 / 相比 / 墊底 / 
 / 香港 /  2.9% / 成長率 / 韓國 /  3% / 新加坡 /   / 3.9% / 相比 /  2016 /  年
 / 回顧 /  2015 /   / 年 / 經濟 / 成長率 /  0.75% /  2014 /   / 年 / 負成長 /   / 2.68% / 差
 / 
 / 總體 / 經濟 / 觀察 / 整 / 進出口 / 狀況 /  12 /   / 黑 / 出口 / 金額 / 年增率 / 
 / 財政部 /  2 月 / 發布 /  1 月 / 進出口 / 數字 /  1 月 / 出口 / 去年 / 衰退 /  13% /  1 月 / 進口 / 
 / 數字 / 前年 / 相比 / 衰退 /  11.7% / 總體 / 經濟 / 進出口 / 狀況 / 請問 / 挽救 / 
 / 產業 / 體質 / 因應 / 整 / 國內外 / 情勢 / 徹頭徹尾 / 做 / 轉型 / 
 / 國外 / 通路 / 加入 /   / TPP / 外交 / 因素 / 加入 /   / RCEP / 體質 / 
 / 國外 / 競爭 / 打掉 / 國內 / 產業 / 體質 / 經濟 / 成長 / 
 / 攸關 / 產業 / 發展 / 蔡英文總統 / 當選人 / 提出 / 產業 / 發展 / 重心 / 
 / 產業 / 贅述 / 裡面 / 科技 / 資通 / 產業 / 生活 /

###  Read data from txt file, and tokenize words, clean-up text

In [4]:
from gensim import corpora, models, utils
from gensim.models import LdaModel, CoherenceModel
#Read data from txt file, and tokenize words, clean-up text
#Creat dictionary for corpus語料庫
import numpy
stayed_lst_ly_new = [d.split() for d in stayed_lst_ly]
dict = corpora.Dictionary(numpy.asarray(stayed_lst_ly_new))
print("dict = ", dict)
print("------------------------------")
#Creat corpus need for Topic Modeling
corpus = [dict.doc2bow(text) for text in stayed_lst_ly_new]   
print("corpus[:1] = ", corpus[:1])   
print("------------------------------")

print("print lists of [word:frequency]")
[[print((dict[id], freq)) for id,freq in cp] for cp in corpus[:5]]
print("======================================================")


unable to import 'smart_open.gcs', disabling that module


dict =  Dictionary(12037 unique tokens: ['林委員岱樺', '列席', '官員', '位', '同仁']...)
------------------------------
corpus[:1] =  [[(0, 1)]]
------------------------------
print lists of [word:frequency]
('林委員岱樺', 1)
('列席', 1)
('官員', 1)
('位', 1)
('同仁', 1)


### Building the Topic Model from LdaModel

In [5]:
lda_model = models.ldamodel.LdaModel(corpus=corpus, id2word= dict, num_topics=10, passes=20, alpha='auto', chunksize=50, per_word_topics=True)
# print(ldamodel.print_topics())   列出最重要的前若干个主题

output = open("林岱樺_LDA_TopicNum10.txt", 'w', encoding='utf-8-sig')#這邊檔名要改
lstTM = lda_model.print_topics(num_topics=10, num_words=20)
for tupleTM in lstTM:
        str1 = "".join(str(x) for x in tupleTM)
        #print("type(str1) = ", type(str1))
        str1 += "\n"
        print("strTM = ", str1)
        output.write(str1)
output.close()
 
print("------------------------------")
print("Perplexity = ", lda_model.log_perplexity(corpus))

print("------------------------------")
cm = CoherenceModel(model=lda_model, texts=list(stayed_lst_ly_new), dictionary=dict, coherence= 'c_v' )
coher_lda = cm.get_coherence()
print("\n Coherence Score: ", coher_lda)
playsound("sound.mp3")#22分鐘

strTM =  00.319*"年" + 0.073*"位" + 0.044*"件" + 0.020*"5" + 0.018*"第" + 0.018*"預算" + 0.000*"僱" + 0.000*"約聘" + 0.000*"勞退新制" + 0.000*"勞動基準法" + 0.000*"定期" + 0.000*"約僱" + 0.000*"聘用" + 0.000*"勞基法" + 0.000*"函釋" + 0.000*"600億" + 0.000*"事務性" + 0.000*"客車" + 0.000*"能傑" + 0.000*"加班"

strTM =  10.317*"本席" + 0.081*"目前" + 0.071*"經濟部" + 0.045*"2" + 0.007*"經濟" + 0.000*"勞退新制" + 0.000*"定期" + 0.000*"加班" + 0.000*"風場" + 0.000*"600億" + 0.000*"長遠" + 0.000*"上限" + 0.000*"勞動基準法" + 0.000*"客車" + 0.000*"函釋" + 0.000*"事務性" + 0.000*"接近" + 0.000*"務" + 0.000*"簡易性" + 0.000*"能傑"

strTM =  20.085*"狀況" + 0.000*"改姓" + 0.000*"收集" + 0.000*"自用車" + 0.000*"職業用" + 0.000*"吊銷" + 0.000*"執照" + 0.000*"改名" + 0.000*"出境" + 0.000*"第十五" + 0.000*"潛逃" + 0.000*"空窗期" + 0.000*"傳輸" + 0.000*"移民署" + 0.000*"戶政" + 0.000*"換發" + 0.000*"姓名" + 0.000*"懇請" + 0.000*"駕駛" + 0.000*"判刑"

strTM =  30.000*"第十五" + 0.000*"吊銷" + 0.000*"生質能" + 0.000*"執照" + 0.000*"自用車" + 0.000*"判刑" + 0.000*"駕駛" + 0.000*"改姓" + 0.000*"熱值" + 0.000*"潛逃" + 0.000*"空窗期" + 0.000*"傳輸" + 0.000*"

ModuleNotFoundError: No module named 'gi'

#### 政見

In [6]:
#h
politics= pd.read_csv('林岱樺政見.txt',encoding="utf-8-sig") #匯該立委立院發言資料
politics

,林岱樺長年在國會經濟委員會，重視大寮、林園、仁武、鳥松經濟發展，林岱樺認為只有高雄好，台灣才會更好，不管大小地 方事，林岱樺都為您做到最好。 一、『交通』－讓城鄉沒有距離：推動捷運林園延伸線、鳳松輕軌捷運線。推動區域交通平衡發展，縮小城鄉差距，促進地區繁榮 二、『教育』－贏在起跑點的下一代：改善教育環境，提升教學設備，促進區域教學品質及平衡教學資源分配。 三、『長照』－普及、平價、優質的長期照護系統：促進政府推動各區老人日托、扶助老人社區參與及補助計畫，以因應日趨嚴重的人口老化問題。 四、『就業』－人人安薪樂業的生活：推動廠商優先錄用在地人才，促進地方就業機會。鼓勵新住民參與並融入社區活動，督促政府重視族群融合，成就平等就業機會。 五、『民生』－打造宜居城市：改善各區排水系統，務必在雨季避免再有積水及淹水的狀況，影響居住安全與品質。增進社區公園開發，鼓勵社區營造，提升居民活動空間，發展優質生活環境。六、『產業』－在地連結，打造高雄產業聚落：(一)仁大澄觀新區開發。(二)重塑高屏溪山水遊廊。(三)改善工業生產設備，減低污染。(四)打造鳳山丘陵南段風景特定區。(五)『和發產業園區』打造為南台灣綠能新興產業廊帶。(六)持續完善群眾募資法，增加中小企業募集資金管道。


In [8]:
str_str= list(politics)
str_str =(','.join('%s' %id for id in str_str)) 
# 開始斷詞
# sentence_segmentation = True,  => To consider delimiters
# segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
# recommend_dictionary = dictionary1, # words in this dictionary are encouraged
# coerce_dictionary = dictionary2, # words in this dictionary are forced
strCut = ws([str_str], sentence_segmentation=True,
                 segment_delimiter_set = {'"',"\r\n","，","「","」",",","？",'\n', "。", ":", "?", "!", ";", "、"},
                 coerce_dictionary = dictionary)

print("------------CKIPTagger Word Segment ws(str,....) + 刪除標點及停用字 ---------------")
stayed_lst= []
for word in strCut[0]:
     # 刪除停用字
    if word not in stopword_list:
        stayed_lst.append(word)
        
print(" / ".join(stayed_lst))
import numpy
stayed_lst_new = [d.split() for d in stayed_lst]
dict = corpora.Dictionary(numpy.asarray(stayed_lst_new))
print("dict = ", dict)
print("------------------------------")
corpus = [dict.doc2bow(text) for text in stayed_lst_new]   
print("corpus[:1] = ", corpus[:1])   
print("------------------------------")

print("print lists of [word:frequency]")
[[print((dict[id], freq)) for id,freq in cp] for cp in corpus[:5]]
print("======================================================")
#playsound("sound.mp3")

------------CKIPTagger Word Segment ws(str,....) + 刪除標點及停用字 ---------------
林岱樺 / 長年 / 國會 / 經濟 / 委員會 / 重視 / 大寮 / 林園 / 仁武 / 鳥松 / 經濟發展 / 林岱樺 / 高雄 / 台灣 / 大小 /   / 事 / 林岱樺 /   / 交通 / 城鄉 / 距離 / 推動 / 捷運 / 林園 / 延伸線 / 鳳松 / 輕軌 / 捷運 / 線 / 推動 / 區域 / 交通 / 平衡 / 發展 / 縮小 / 城鄉差距 / 地區 / 繁榮 /  二 / 教育 / 贏 / 起跑點 / 代 / 改善 / 教育 / 環境 / 提升 / 教學 / 設備 / 區域 / 教學 / 品質 / 平衡 / 教學 / 資源 / 分配 /   / 長照 / 普及 / 平價 / 優質 / 長期照護 / 系統 / 政府 / 推動 / 區 / 老人 / 日托 / 扶助 / 老人 / 社區 / 參與 / 補助 / 計畫 / 因應 / 日趨 / 人口 / 老化 /   / 就業 / 安薪樂業 / 生活 / 推動 / 廠商 / 優先 / 錄用 / 在地 / 人才 / 地方 / 就業 / 機會 / 鼓勵 / 新住民 / 參與 / 融入 / 社區 / 活動 / 督促 / 政府 / 重視 / 族群 / 融合 / 成就 / 平等 / 就業 / 機會 /  五 / 民生 / 打造 / 宜居 / 城市 / 改善 / 區 / 排水 / 系統 / 務必 / 雨季 / 積水 / 淹水 / 狀況 / 影響 / 居住 / 品質 / 增進 / 社區公園 / 開發 / 鼓勵 / 社區營造 / 提升 / 居民 / 活動 / 空間 / 發展 / 優質 / 生活 / 環境 / 產業 / 』－ / 在地 / 連結 / 打造 / 高雄 / 產業 / 聚落 / 仁大 / 澄觀 / 新 / 區 / 開發 / (二) / 重塑 / 高屏溪 / 山水 / 遊廊 / (三) / 改善 / 工業 / 生產 / 設備 / 減低 / 污染 / (四) / 打造 / 鳳山丘陵 / 南段 / 風景 / 特定區 / (五) / 和發產業園區 / 打造為 / 南台灣 / 綠能 / 新興 / 產業 / 廊帶 / (六) / 持續 / 完善 / 群眾 / 募資

In [9]:
#h
lda_model_p = models.ldamodel.LdaModel(corpus=corpus, id2word= dict, num_topics=8, passes=20, alpha='auto', chunksize=50, per_word_topics=True)
# print(ldamodel.print_topics())   列出最重要的前若干个主题

output = open("林岱樺政見_LDA_TopicNum8.txt", 'w', encoding='utf-8-sig')#這邊檔名要改
lstTM = lda_model_p.print_topics(num_topics=8, num_words=20)
for tupleTM in lstTM:
        #str1 = "".join(str(x) for x in tupleTM)
        for x in tupleTM:
                str1 = "".join(str(x))
        #print("type(str1) = ", type(str1))
        str1 += "\n"
        print("strTM = ", str1)
        output.write(str1)
output.close()
 
print("------------------------------")
print("Perplexity = ", lda_model_p.log_perplexity(corpus))

print("------------------------------")
cm = CoherenceModel(model=lda_model_p, texts=list(stayed_lst_new), dictionary=dict, coherence= 'c_v' )
coher_lda = cm.get_coherence()
print("\n Coherence Score: ", coher_lda)
#playsound("sound.mp3")#14分鐘

strTM =  0.121*"打造" + 0.063*"林岱樺" + 0.059*"完善" + 0.059*"遊廊" + 0.055*"發展" + 0.048*"區域" + 0.023*"經濟" + 0.023*"起跑點" + 0.023*"委員會" + 0.023*"地區" + 0.004*"』－" + 0.004*"居住" + 0.004*"連結" + 0.004*"增進" + 0.004*"社區公園" + 0.004*"空間" + 0.004*"雨季" + 0.004*"居民" + 0.004*"開發" + 0.004*"社區營造"

strTM =  0.051*"高雄" + 0.040*"林園" + 0.033*"聚落" + 0.033*"務必" + 0.033*"居民" + 0.033*"社區營造" + 0.033*"積水" + 0.033*"居住" + 0.033*"融合" + 0.033*"成就" + 0.033*"宜居" + 0.033*"城市" + 0.033*"排水" + 0.033*"族群" + 0.022*"代" + 0.022*"縮小" + 0.022*"繁榮" + 0.004*"錄用" + 0.004*"廠商" + 0.004*"增進"

strTM =  0.089*"推動" + 0.056*"山水" + 0.056*"持續" + 0.056*"南段" + 0.056*"綠能" + 0.056*"特定區" + 0.052*"提升" + 0.034*"增進" + 0.022*"國會" + 0.022*"延伸線" + 0.022*"事" + 0.004*"督促" + 0.004*"老化" + 0.004*"融入" + 0.004*"新住民" + 0.004*"高屏溪" + 0.004*"人才" + 0.004*"融合" + 0.004*"重塑" + 0.004*"日趨"

strTM =  0.052*"區" + 0.048*"就業" + 0.034*"在地" + 0.034*"活動" + 0.034*"鼓勵" + 0.034*"品質" + 0.034*"系統" + 0.034*"優質" + 0.031*"環境" + 0.030*"老人" + 0.030*"參與" + 0.030*"社區" + 0.030*"政府" + 0.016*"老

In [10]:
#h
from collections import Counter
import pandas as pd
topic_politics=pd.read_csv('純文字林岱樺政見_LDA_TopicNum8.csv',encoding="utf-8-sig")
topic_ly=pd.read_csv('純文字林岱樺_LDA_TopicNum10.csv',encoding="utf-8-sig")
topic_politics=topic_politics.loc[0:,].drop_duplicates().values.tolist()
topic_ly=topic_ly.loc[0:,].drop_duplicates().values.tolist()
from tkinter import _flatten
topic_politics=list(_flatten(topic_politics))
topic_ly=list(_flatten(topic_ly))
Counter(topic_politics+topic_ly)

Counter({'林岱樺': 1,
         '完善': 1,
         '遊廊': 1,
         '發展': 1,
         '區域': 1,
         '經濟': 2,
         '起跑點': 1,
         '委員會': 1,
         '地區': 1,
         '居住': 1,
         '連結': 1,
         '增進': 1,
         '社區公園': 1,
         '空間': 1,
         '雨季': 1,
         '居民': 1,
         '開發': 1,
         '社區營造': 1,
         '高雄': 1,
         '林園': 1,
         '聚落': 1,
         '務必': 1,
         '積水': 1,
         '融合': 1,
         '成就': 1,
         '宜居': 1,
         '城市': 1,
         '排水': 1,
         '族群': 1,
         '代': 1,
         '縮小': 1,
         '繁榮': 1,
         '錄用': 1,
         '廠商': 1,
         '推動': 1,
         '山水': 1,
         '持續': 1,
         '南段': 1,
         '綠能': 1,
         '特定區': 1,
         '提升': 1,
         '國會': 1,
         '延伸線': 1,
         '事': 1,
         '督促': 1,
         '老化': 1,
         '融入': 1,
         '新住民': 1,
         '高屏溪': 1,
         '人才': 1,
         '重塑': 1,
         '日趨': 1,
         '區': 1,
         '就業': 1,
         '在地': 1,
  

In [ ]:
#以下計算相關程度

In [11]:
from sklearn.feature_extraction.text import CountVectorizer  
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(stayed_lst_ly) # 立法院, return a matrix (#of corpus) x (# of word) # 201 x 4835
word = vectorizer.get_feature_names() # column name which is the word
message_column = word
message_column

['000',
 '007',
 '009',
 '01',
 '02',
 '03',
 '037',
 '04',
 '045',
 '049',
 '05',
 '06',
 '08',
 '080',
 '0823',
 '088',
 '10',
 '100',
 '1000',
 '101',
 '102',
 '103',
 '104',
 '104年',
 '105',
 '106',
 '1060',
 '107',
 '108',
 '108年',
 '109',
 '10日',
 '11',
 '110',
 '110萬',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '117萬',
 '119',
 '11日',
 '12',
 '120',
 '122',
 '123',
 '127',
 '13',
 '130',
 '131',
 '132',
 '135億',
 '139',
 '14',
 '140',
 '141',
 '143',
 '145',
 '145億',
 '15',
 '150',
 '16',
 '160',
 '161',
 '164',
 '167',
 '169',
 '17',
 '173',
 '175',
 '176',
 '179',
 '18',
 '180',
 '184',
 '186',
 '19',
 '192',
 '194',
 '1947',
 '195',
 '1950',
 '1960',
 '1967',
 '1984',
 '1987',
 '1990',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '20',
 '200',
 '2000',
 '2002',
 '2003',
 '2005',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2011年',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2023',
 '2024',
 '2025',
 '20

In [12]:
#politics_word
match_column = list(set(stayed_lst).intersection(vectorizer.get_feature_names()))
match_column

['平等',
 '台灣',
 '活動',
 '教育',
 '資源',
 '務必',
 '人才',
 '平價',
 '經濟',
 '平衡',
 '重視',
 '計畫',
 '雨季',
 '鳥松',
 '優質',
 '在地',
 '推動',
 '林岱樺',
 '減低',
 '成就',
 '繁榮',
 '系統',
 '族群',
 '增進',
 '排水',
 '仁武',
 '因應',
 '淹水',
 '新住民',
 '工業',
 '扶助',
 '中小企業',
 '委員會',
 '發展',
 '廠商',
 '日趨',
 '社區',
 '優先',
 '生活',
 '生產',
 '大小',
 '募資',
 '大寮',
 '管道',
 '城鄉',
 '分配',
 '狀況',
 '地區',
 '政府',
 '督促',
 '打造',
 '設備',
 '距離',
 '污染',
 '區域',
 '提升',
 '城市',
 '持續',
 '新興',
 '鼓勵',
 '捷運',
 '人口',
 '老化',
 '城鄉差距',
 '老人',
 '補助',
 '國會',
 '交通',
 '開發',
 '聚落',
 '長照',
 '綠能',
 '參與',
 '高屏溪',
 '機會',
 '融入',
 '改善',
 '居民',
 '空間',
 '完善',
 '教學',
 '民生',
 '特定區',
 '產業',
 '長年',
 '連結',
 '居住',
 '影響',
 '經濟發展',
 '普及',
 '林園',
 '縮小',
 '就業',
 '品質',
 '資金',
 '高雄',
 '環境',
 '地方',
 '南台灣']

In [13]:
#立法院
document_term_ly = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names())
#document_term_ly 

In [14]:
match_document_politics = document_term_ly[match_column]
match_document_politics

,平等,台灣,活動,教育,資源,務必,人才,平價,經濟,平衡,...,普及,林園,縮小,就業,品質,資金,高雄,環境,地方,南台灣
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106606,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
106607,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
106608,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
106609,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
match_document_politics["sum"] = match_document_politics.sum(axis=1)
match_document_politics

/home/abner0908/anaconda2/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,平等,台灣,活動,教育,資源,務必,人才,平價,經濟,平衡,...,林園,縮小,就業,品質,資金,高雄,環境,地方,南台灣,sum
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106606,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
106607,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
106608,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
106609,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
sum(match_document_politics["sum"].tolist()) / len(match_column) 

50.75757575757576

In [17]:
sum(match_document_politics["sum"].tolist()) / len(match_column) /len(zhao) # 相關度

0.3965435606060606

臉書

In [18]:
fb = pd.read_csv('nysu_10902_2019_research_right.csv')
fb = fb[fb.page_name == "林岱樺"]
fb

,post_id,page_id,page_name,page_category,message,story,type,permalink,created_time_gmt,created_time_taipei,...,link_name,link_description,like_count,haha_count,love_count,angry_count,sad_count,wow_count,comment_count,share_count
12960,139678436043353_2315740748437100,139678436043353,林岱樺,政治人物,一把火燒起的不只是工安！\n\n昨晚的一把巨火！\n再度燃點出咱林園的哀愁！\n\n居民們無...,NaN,photo,https://www.facebook.com/139678436043353/posts...,2019-01-15T12:30:00+0000,2019-01-15T20:30:00,...,NaN,NaN,100,0,0,0,0,0,10,8
12961,139678436043353_2315296331814875,139678436043353,林岱樺,政治人物,14日20時48分林園區北汕路26號發生火警，\n\n岱樺接獲反應後，馬上向有關單位查證獲報...,NaN,video,https://www.facebook.com/139678436043353/posts...,2019-01-15T04:07:27+0000,2019-01-15T12:07:27,...,NaN,NaN,71,0,0,0,0,0,1,3
12962,139678436043353_2314352681909240,139678436043353,林岱樺,政治人物,14日20時48分林園區北汕路26號發生火警，\n\n岱樺接獲反應後，馬上向有關單位查證獲報...,NaN,photo,https://www.facebook.com/139678436043353/posts...,2019-01-14T13:37:49+0000,2019-01-14T21:37:49,...,NaN,NaN,83,0,0,0,0,0,4,5
12963,139678436043353_2313063145371527,139678436043353,林岱樺,政治人物,【歲末年終，圍爐迎新年】\n\n隨著農曆春節的靠近，岱樺昨天參與了仁武通法寺的新春圍爐活動，...,NaN,photo,https://www.facebook.com/139678436043353/posts...,2019-01-14T02:00:01+0000,2019-01-14T10:00:01,...,NaN,NaN,131,0,0,0,0,0,3,8
12964,139678436043353_2310100419001133,139678436043353,林岱樺,政治人物,【2513公斤過期奶精可以做幾杯手搖杯？】\n\n日前高雄檢調破獲黑心奶精公司，將過期及即期...,NaN,link,https://www.facebook.com/139678436043353/posts...,2019-01-12T03:00:00+0000,2019-01-12T11:00:00,...,黑心奶精2513公斤流入市面 負責人10萬交保 | 社會 | 重點新聞 | 中央社 CNA,高雄一公司涉將過期奶精改標再賣，高雄檢調起出有疑慮奶精共8441公斤，包括流入市面2513公...,187,0,0,0,0,0,13,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13317,139678436043353_2957649860912849,139678436043353,林岱樺,政治人物,早安～\n明天12/30(ㄧ)上午11:00-11:30\n「辣台號」要開來我們鳥松和岱樺ㄧ...,NaN,photo,https://www.facebook.com/139678436043353/posts...,2019-12-29T00:31:23+0000,2019-12-29T08:31:23,...,NaN,NaN,690,0,0,0,0,0,29,11
13318,139678436043353_2956416984369470,139678436043353,林岱樺,政治人物,謝謝今天 潘孟安縣長特別來到林園陪同岱樺掃街拜票，也很謝謝鄉親朋友的支持和鼓勵。\n\n我們...,NaN,photo,https://www.facebook.com/139678436043353/posts...,2019-12-28T11:35:11+0000,2019-12-28T19:35:11,...,NaN,NaN,1902,0,0,0,0,0,50,27
13319,139678436043353_2955576204453548,139678436043353,林岱樺,政治人物,早安～\n\n原先高雄市民所享有澄清湖風景區免費入園的福利優惠，僅到今年的9月份為止。為了能...,NaN,photo,https://www.facebook.com/139678436043353/posts...,2019-12-28T01:18:39+0000,2019-12-28T09:18:39,...,NaN,NaN,977,0,0,0,0,0,56,21
13320,139678436043353_2954172724593896,139678436043353,林岱樺,政治人物,今天天氣轉冷了，大家記得要穿暖和一點哦！\n\n12/7總部成立大會的影片已經剪輯完畢，請大...,NaN,video,https://www.facebook.com/139678436043353/posts...,2019-12-27T11:07:52+0000,2019-12-27T19:07:52,...,NaN,NaN,1350,0,0,0,0,0,62,23


In [19]:
str_str= list(fb['message'])
str_str =(','.join('%s' %id for id in str_str)) 
# 開始斷詞
# sentence_segmentation = True,  => To consider delimiters
# segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
# recommend_dictionary = dictionary1, # words in this dictionary are encouraged
# coerce_dictionary = dictionary2, # words in this dictionary are forced
strCut = ws([str_str], sentence_segmentation=True,
                 segment_delimiter_set = {"🎉","📢","✨","👉",'"',"\r\n","，","「","」",",","？",'\n', "。", ":", "?", "!", ";", "、"},
                 coerce_dictionary = dictionary)

print("------------CKIPTagger Word Segment ws(str,....) + 刪除標點及停用字 ---------------")
stayed_lst_fb= []
for word in strCut[0]:
     # 刪除停用字
    if word not in stopword_list:
        stayed_lst_fb.append(word)
        
print(" / ".join(stayed_lst_fb))
#playsound("sound.mp3")#五分鐘

------------CKIPTagger Word Segment ws(str,....) + 刪除標點及停用字 ---------------
火 / 燒 / 工安 / 
 / 
 / 昨晚 / 巨火 / 
 / 再度 / 燃點出 / 林園 / 哀愁 / 
 / 
 / 居民 / 無奈 / 告訴 / 岱樺
 / 火 / 奧運 / 聖火 / 發爐 / 光明 / 光明燈 / 
 / 岱樺 / 字字 / 句句 / 間 / 年 / 累積 / 哀述 / 
 / 工安 / 做 / 難 / 
 / 
 / 火 / 燒 / 哀愁 / 中油 / 態度 / 
 / 態度 / 
 / 工安 / 
 / 
 / 岱樺 / 倔強 / 態度 / 盯緊 / 部會 / 工廠 / 
 / 守護 / 林園 / 14日 / 20時 / 48分 / 林園區 / 北汕路 / 26號 / 發生 / 火警 / 
 / 
 / 岱樺 / 接獲 / 單位 / 查證 / 獲報 / 
 / 現場 / 係 / 中油 / 林園 / 廠 / 新 / 三輕 / 工廠 / 電力 / 系統 / 故障 / 跳機 / 管道 / 乙烯 / 碳氫化合物 / 排 / 燃燒塔 / 燃燒 / 中油 / 製程 / 流程 / 現場 / 危害 / 虞 / 受困 / 受傷 / 
 / 
 / 新 / 狀況 / 持續 / 粉絲頁 / 更新 / 
 / 
 / --
 / 
 / 結報 /   / 林園工業區服務中心 / 災情 / 通報 / 
 / 災情 / 描述 / 108年 / 1月 / 14日 / 20時 / 51分 / 林園工業區 / 台灣中油公司 / 石化 / 事業部 / 新 / 三輕 / 泵浦 / 停電 / 導致 / 系統 / 異常 / 啟動 / 機置 / 導致 / 北區 / 燃燒塔 / 排放 / 大火 / 謝 / 區長 / 水福 / 王耀裕議員 / 林岱樺 / 立委 / 助理 / 邱于軒議員 / 五福 / 溪州里 / 約 / 位 / 里長 / 民眾 / 約 / 50 / 位 / 到場 / 關切 / 媒體 / 人員 / 傷亡 / 
 / 
 / 搶救 / 情形 / 監測 / 中心 / 人員 / 中油 / 門 / 口 / 鄰近 / 處 / 採樣 / 作業 / 林園 / 消防 / 車 / 台 / 救護車 / 台 / 消防 / 人員 / 11

In [21]:
import numpy
stayed_lst_fb_new = [d.split() for d in stayed_lst_fb]
dict = corpora.Dictionary(numpy.asarray(stayed_lst_fb_new))
print("dict = ", dict)
print("------------------------------")
#Creat corpus need for Topic Modeling
corpus_fb = [dict.doc2bow(text) for text in stayed_lst_fb_new]   
print("corpus[:1] = ", corpus[:1])   
print("------------------------------")

print("print lists of [word:frequency]")
[[print((dict[id], freq)) for id,freq in cp] for cp in corpus_fb[:5]]
print("======================================================")

lda_model = models.ldamodel.LdaModel(corpus=corpus_fb, id2word= dict, num_topics=5, passes=20, alpha='auto', chunksize=50, per_word_topics=True)
# print(ldamodel.print_topics())   列出最重要的前若干个主题

output = open("林岱樺fb_LDA_TopicNum5.txt", 'w', encoding='utf-8-sig')#這邊檔名要改
lstTM = lda_model.print_topics(num_topics=5, num_words=20)
for tupleTM in lstTM:
        str1 = "".join(str(x) for x in tupleTM)
        #print("type(str1) = ", type(str1))
        str1 += "\n"
        print("strTM = ", str1)
        output.write(str1)
output.close()
 
print("------------------------------")
print("Perplexity = ", lda_model.log_perplexity(corpus_fb))

print("------------------------------")
cm = CoherenceModel(model=lda_model, texts=list(stayed_lst_fb_new), dictionary=dict, coherence= 'c_v' )
coher_lda = cm.get_coherence()
print("\n Coherence Score: ", coher_lda)
playsound("sound.mp3")

dict =  Dictionary(6538 unique tokens: ['火', '燒', '工安', '昨晚', '巨火']...)
------------------------------
corpus[:1] =  [[(0, 1)]]
------------------------------
print lists of [word:frequency]
('火', 1)
('燒', 1)
('工安', 1)
strTM =  00.030*"活動" + 0.028*"新聞" + 0.023*"外" + 0.007*"兩" + 0.006*"#1" + 0.005*"年" + 0.005*"直播" + 0.004*"早安" + 0.004*"做對" + 0.004*"掃街" + 0.004*"拷潭" + 0.004*"投" + 0.003*"優惠" + 0.003*"排水" + 0.003*"辣台號" + 0.003*"投票" + 0.002*"1/5" + 0.002*"協商" + 0.002*"候選人" + 0.002*"11"

strTM =  10.242*"岱樺" + 0.029*"立委" + 0.003*"#1" + 0.003*"直播" + 0.003*"拷潭" + 0.003*"掃街" + 0.002*"優惠" + 0.002*"早安" + 0.002*"投" + 0.002*"做對" + 0.002*"排水" + 0.002*"辣台號" + 0.002*"1/5" + 0.002*"協商" + 0.002*"11" + 0.002*"持續" + 0.002*"園" + 0.002*"候選人" + 0.002*"潘孟安" + 0.002*"投票"

strTM =  20.097*"#" + 0.022*"林園" + 0.021*"改善" + 0.016*"努力" + 0.015*"鄉親" + 0.013*"工程" + 0.012*"民主" + 0.012*"成功" + 0.010*"天" + 0.010*"#1" + 0.010*"位" + 0.009*"仁武" + 0.008*"時間" + 0.008*"特別" + 0.008*"資源" + 0.008*"做對" + 0.007*"成立" + 0.007*"發展" + 0

ModuleNotFoundError: No module named 'gi'

In [22]:
from collections import Counter
import pandas as pd
topic_fb=pd.read_csv('純文字林岱樺fb_LDA_TopicNum5.csv',encoding="utf-8-sig")
topic_fb=topic_fb.loc[0:,].drop_duplicates().values.tolist()
from tkinter import _flatten
topic_fb=list(_flatten(topic_fb))
Counter(topic_politics+topic_fb)

from sklearn.feature_extraction.text import CountVectorizer  
vectorizer = CountVectorizer()
fb_X = vectorizer.fit_transform(stayed_lst_fb) # fb return a matrix (#of corpus) x (# of word) # 201 x 4835
word = vectorizer.get_feature_names() # column name which is the word
message_column = word
message_column
#fb
document_term_fb = pd.DataFrame(fb_X.toarray(), columns = vectorizer.get_feature_names())
document_term_fb


,00,000,02,039,03分,0420,06,07,08,0800,...,龍舟,龍蟠,龐大,龔厝,龔厝橋,１號,１４,５月,６日,ｙａｈｏｏ
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27735,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27736,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27737,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27738,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
match_column = list(set(stayed_lst).intersection(vectorizer.get_feature_names()))
match_column
match_document_politics = document_term_fb[match_column]
match_document_politics["sum"] = match_document_politics.sum(axis=1)
match_document_politics
sum(match_document_politics['sum'].tolist()) / len(match_column) #在趙天麟的所有篇中會有幾篇有包含政策相關的

/home/abner0908/anaconda2/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


23.92391304347826

In [24]:
match_document_politics

,台灣,活動,教育,資源,務必,人才,經濟,平衡,重視,計畫,...,風景,林園,就業,品質,資金,高雄,環境,地方,南台灣,sum
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27735,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27736,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
27737,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27738,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
#政見跟臉書相關度
sum(match_document_politics['sum'].tolist()) / len(match_column) /len(fb) # 相關度

0.06608815757866923

In [34]:
Counter(topic_ly+topic_fb)
fb_X = vectorizer.fit_transform(stayed_lst_fb) # fb return a matrix (#of corpus) x (# of word) # 201 x 4835
word = vectorizer.get_feature_names() # column name which is the word
message_column = word
message_column
#fb
document_term_fb = pd.DataFrame(fb_X.toarray(), columns = vectorizer.get_feature_names())
document_term_fb

,00,000,0006,004,006億,008,01,0100,02,0217161130,...,齊聚,龍成宮,龍捲風,龍湖廟,龍頭,龍鳳,龐大,龜裂,理念,精神
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55630,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55631,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55632,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55633,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
match_column = list(set(stayed_lst_ly).intersection(vectorizer.get_feature_names()))
match_column
match_document_politics = document_term_fb[match_column]
match_document_politics["sum"] = match_document_politics.sum(axis=1)
match_document_politics
sum(match_document_politics['sum'].tolist()) / len(match_column) 

/home/abner0908/anaconda2/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


5.48169014084507

In [28]:
#政見跟臉書相關度
sum(match_document_politics['sum'].tolist()) / len(match_column) /len(fb) # 相關度

0.015142790444323399

In [30]:
del document_term_ly
del match_document_politics
del fb
del zhao
gc.collect()

NameError: name 'document_term_ly' is not defined